In [66]:
import tweepy
import os
from dotenv import load_dotenv
import datetime as dt
import re

In [6]:
load_dotenv()
consumer_key = os.getenv('TWITTER_CONSUMER_KEY')
consumer_secret = os.getenv('TWITTER_CONSUMER_SECRET')
access_token = os.getenv('TWITTER_TOKEN_KEY')
access_token_secret = os.getenv('TWITTER_TOKEN_SECRET')

In [10]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_token_secret)

In [11]:
api = tweepy.API(auth)

In [38]:
screen_name="VaxHuntersCAN"

In [47]:
tweets = api.user_timeline(screen_name, include_rts=True, count=200, tweet_mode = 'extended')

In [52]:
now = dt.datetime.now()
time_36h = now - dt.timedelta(hours=36)

In [55]:
# extract tweets from last 36hrs
tweets_36h = []
for tweet in tweets:
    if tweet.created_at > time_36h:
        tweets_36h.append(tweet)
    else:
        print('Old tweet')

Old tweet
Old tweet
Old tweet
Old tweet
Old tweet
Old tweet
Old tweet
Old tweet
Old tweet
Old tweet
Old tweet
Old tweet
Old tweet
Old tweet
Old tweet
Old tweet
Old tweet
Old tweet
Old tweet
Old tweet
Old tweet
Old tweet
Old tweet
Old tweet
Old tweet
Old tweet
Old tweet
Old tweet
Old tweet
Old tweet
Old tweet


In [60]:
# number tweets from last 36h
len(tweets_36h)

169

In [154]:
full_text = "[ON] Richmond Hill 40+ APPOINTMENTS FROM 10am-5:30pm\nShoppers Drug Mart - Oakridges: 13070 Yonge Street\n22 AstraZeneca doses available, Online Booking"
lines = full_text.split("\n")
age_pat = re.compile(r"(street|road|drive|boulevard|avenue)", re.IGNORECASE)
for line in lines:
    age = age_pat.search(line)
    if age:
        street = age.group(0)
        sub = st_lookup[street.lower()]
        line = re.sub(street, sub, line)
        print(line)

Shoppers Drug Mart - Oakridges: 13070 Yonge st


In [153]:
st_lookup = {'street': 'st', 'road': 'rd', 'drive': 'dr', 'boulevard': 'blvd', 'avenue':'ave'}

In [189]:
a = []
if(a):
    print(a[0])

In [270]:
class TweetExtract(object):
    
    def __init__(self):
        self.age_pat = re.compile(r"(\d{2})\+")
        self.year_pat = re.compile(r"(Born|born).*(19\d{2})")
        self.postal_pat = re.compile(r"[A-Z]\d[A-Z]")
        self.address_pat = re.compile(r".*(\d+.*\s(?:st|rd|dr|ave|blvd|hwy|edge))(\s|\W|\b)", re.IGNORECASE)
        self.sub_addr_pat = re.compile(r"(?:\d+\s*[\-]*)+(?:\s\w+)*\s(?:st|rd|dr|ave|blvd|hwy)(?:(?:[\s\.]\d+)|(?:,(?:\s*\w+)*)+)?", re.IGNORECASE)
        self.street_pat = re.compile(r"(street|road|drive|boulevard|avenue|highway)", re.IGNORECASE)
        self.st_lookup = {'street': 'st', 'road': 'rd', 'drive': 'dr', 'boulevard': 'blvd', 'avenue':'ave', 'highway':'hwy'}
        self.num_match = 0
        
        # load postal codes
        self._p = pickle.load(open('postal_index.p', "rb"))
        
        postal_codes = pd.read_csv('./CanadianPostalCodes.csv')
        self._fsa = np.asarray(postal_codes.loc[:, ['FSA']])
        del postal_codes
        
    def get_age(self, full_text):
        extracted_age = None
        age = self.age_pat.search(full_text)
        year = self.year_pat.search(full_text)
        
        if(age): # check for age
            extracted_age = age.group(1)
        elif (year): # check for year if age not included
            extracted_age = 2021 - int(year.group(2))
            
        return extracted_age
    
    def search_address(self, address, sub_addr=None):
        lat = lon = None
        # create parameter string to minimize formatting issues
        payload= {'q': address, 'format': 'jsonv2','countrycodes':'ca','addressdetails':1}
        payload_str = "&".join("%s=%s" % (k,v) for k,v in payload.items())
        resp = requests.get('https://nominatim.openstreetmap.org/search',params=payload_str)
        if resp.status_code == 200:
            try:
                geodata = json.loads(resp.text)
                if(geodata):
                    lat = geodata[0]['lat']
                    lon = geodata[0]['lon']
            except Exception as e:
                pass # do nothing
                
        if(not lat and sub_addr):
            return self.search_address(sub_addr)
        return lat, lon
    
    def find_postals(self, lat, lon):
        labels, distances = self._p.knn_query([[lat, lon]], k = 100)
        nearest = self._fsa[labels[-1]] #find fsa for returned labels
        u, indices = np.unique(nearest, return_index=True)
        i = sorted(indices)[:6] # pick 5 closest unique postal codes
        return nearest[i].flatten()
        
    def get_postal(self, full_text):
        extracted_postal = []
        postals = self.postal_pat.findall(full_text)
        
        if postals: #check for postal code
            extracted_postal = postals
        else: # check for address
            lines = full_text.split("\n")
            for line in lines:
                # replace street endings to standardize
                st_match = self.street_pat.search(line)
                if st_match:
                    street = st_match.group(0)
                    sub = self.st_lookup[street.lower()]
                    line = re.sub(street, sub, line)
               
                # search for address
                match = self.sub_addr_pat.search(line)
                if match:
                    lat,lon = self.search_address(line, match.group(0))
                    
                    if(lat):
                        # extract nearby postal codes
                        extracted_postal = self.find_postals(lat,lon)
        
        return extracted_postal
        
    def single_tweet_extract(self, tweet):
        t_data = dict()
        
        full_text = tweet.full_text

        age = self.get_age(full_text)
        postal_code = self.get_postal(full_text)

        if(age and len(postal_code) > 0):
            t_data['id'] = tweet.id # use id for max_id
            t_data['name'] = tweet.user.name # Vaccine Hunters Canada
            t_data['screen_name'] = tweet.user.screen_name
            t_data['lastUpdated'] = tweet.created_at
            t_data['eligibility'] = full_text
            t_data['postal_code'] = postal_code
            t_data['status'] = "open"
            t_data['age'] = age
            t_data['booking_link'] = '' #TODO: extract booking link
            t_data['phone'] = '' #TODO: extract phone number
        elif(hasattr(tweet, 'quoted_status')):
            return self.single_tweet_extract(tweet.quoted_status)

        return t_data
    
    def extract_tweets(self, tweets):
        valid_tweet_data = []
        for tweet in tweets:
            tweet_data = self.single_tweet_extract(tweet)
            if(tweet_data):
                valid_tweet_data.append(tweet_data)
        print(self.num_match)        
        return valid_tweet_data

In [267]:
extract = TweetExtract()
data = extract.extract_tweets(tweets_36h)

0


In [268]:
len(data)

93

In [269]:
data[:3]

[{'id': 1388795422991396869,
  'name': 'Vaccine Hunters Canada',
  'screen_name': 'VaxHuntersCan',
  'lastUpdated': datetime.datetime(2021, 5, 2, 10, 0, 6),
  'eligibility': '[ON] 18+ RESIDENT OR WORKER IN M3J M3K M3M M3N M6M M9L M9M M9N ONLY\n\nDownsview Arena (1633 Wilson Ave) is OPEN TODAY at 9AM for walk-ins with 1500 doses of vaccine. Proof of address is REQUIRED! #COVID19 #COVID19Vaccine #COVID19ON #vhcON https://t.co/jst1C8sxEa',
  'postal_code': ['M3J', 'M3K', 'M3M', 'M3N', 'M6M', 'M9L', 'M9M', 'M9N'],
  'status': 'open',
  'age': '18',
  'booking_link': '',
  'phone': ''},
 {'id': 1388697630205751297,
  'name': 'Vaccine Hunters Canada',
  'screen_name': 'VaxHuntersCan',
  'lastUpdated': datetime.datetime(2021, 5, 2, 3, 31, 30),
  'eligibility': '[ON] Brampton 40+ \nShoppers Drug Mart 10665 Bramalea Rd, Brampton\nLots of doses available! No wait times!\nWalk-in during open hours or call for appointment 905-793-2011\n\nCats are also our spokesperson for mask wearing #COVID19 #CO

In [140]:
tweet.quoted_status.full_text.split("\n")

['[ON] 18+ M9N M3M M3H M6L M6K (ALL NEW) M3J M3L M3N M9L M9M RESIDENTS ONLY',
 "Oakdale Community Centre is now accepting MORE POSTAL CODES. 350 Grandravine Drive! Proof of address required. Don't make us do commentary.",
 'https://t.co/3ovFYmmuRx #COVID19 #COVID19Vaccine #COVID19ON #vhcON']

In [167]:
tweet.entities['urls']

[{'url': 'https://t.co/xuX7NcrMp5',
  'expanded_url': 'https://twitter.com/VaxHuntersCan/status/1388189057885392905',
  'display_url': 'twitter.com/VaxHuntersCan/…',
  'indices': [47, 70]}]

In [92]:
tweets_36h[1].id < 1388835561713061890

True

In [219]:
len(tweets_36h)

169

# Lambda Fxn

In [261]:
import tweepy
import os
import re
import requests
import json
import time
import pandas as pd
import numpy as np
import pickle

In [ ]:
consumer_key = os.getenv('TWITTER_CONSUMER_KEY')
consumer_secret = os.getenv('TWITTER_CONSUMER_SECRET')
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)

In [ ]:
api = tweepy.API(auth)

In [90]:
# connect to api, query for since id and screen_name to search
since_id = 1388835561713061890

# check for latest tweets
tweets = api.user_timeline(screen_name, include_rts=True, count=200, since_id=since_id, tweet_mode = 'extended')

# check length, and extract if > 0
len(tweets)